In [102]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
from kobert import get_tokenizer
import os
import re

In [103]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
device = torch.device("cuda:3")
print(device)

cuda:3


# 0. 데이터 전처리

### 형태소 분석
* Okt
* Komoran
* Mecab 안깔린다 제거!
* Khaiii 안깔린다 제거!
* Kkma 느리다 제거!


### Okt vs. Komoran
* Okt: 느림, 정확
* Komoran: 빠름, 토큰화가 안되는 경우가 있음
* Okt 쓰고 느리면 Komoran 쓰는걸로~

In [104]:
DATA_PATH = "../dataset"

In [105]:
all_df = pd.read_csv(os.path.join(DATA_PATH, "raw/sentimental_analysis.csv"))
all_df.head(3)

,cat1,cat2,sentence
0,분노,노여워하는,일은 왜 해도 해도 끝이 없을까? 화가 난다.
1,분노,노여워하는,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.
2,분노,노여워하는,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...


In [106]:
X_length = all_df['sentence'].astype(str).apply(len)
print('문장 길이 최대/최소: {} {}'.format(np.max(X_length), np.min(X_length)))
print('문장 길이 평균값: {:.2f}'.format(np.mean(X_length)))
print('문장 길이 표준편차: {:.2f}'.format(np.std(X_length)))
print('문장 길이 중간값: {}'.format(np.median(X_length)))

문장 길이 최대/최소: 298 3
문장 길이 평균값: 34.64
문장 길이 표준편차: 12.74
문장 길이 중간값: 34.0


In [107]:
def preprocessing(str):
    str_preprocessed = re.sub('[^a-zA-Z가-힣ㄱ-ㅎㅏ-ㅣ\s]','',str)
    #str_preprocessed = str_preprocessed.replace(" ", "")
    return str_preprocessed

In [108]:
test_sentences = all_df["sentence"][0:20].map(lambda x: preprocessing(x))
print(test_sentences)

0                               일은 왜 해도 해도 끝이 없을까 화가 난다
1          이번 달에 또 급여가 깎였어 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나
2     회사에 신입이 들어왔는데 말투가 거슬려 그런 애를 매일 봐야 한다고 생각하니까 스트...
3      직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜 일도 많은 데 정말 분하고 섭섭해
4                  얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나
5           직장에 다니고 있지만 시간만 버리는 거 같아 진지하게 진로에 대한 고민이 생겨
6                성인인데도 진로를 아직도 못 정했다고 부모님이 노여워하셔 나도 섭섭해
7                          퇴사한 지 얼마 안 됐지만 천천히 직장을 구해보려고
8              졸업반이라서 취업을 생각해야 하는데 지금 너무 느긋해서 이래도 되나 싶어
9                               요즘 직장생활이 너무 편하고 좋은 것 같아
10                              취업해야 할 나이인데 취업하고 싶지가 않아
11                             면접에서 부모님 직업에 대한 질문이 들어왔어
12    큰일이야 부장님께 결재받아야 하는 서류가 사라졌어 한 시간 뒤에 제출해야 하는데 어...
13    나 얼마 전에 면접 본 회사에서 면접 합격했다고 연락받았었는데 오늘 다시 입사 취소...
14            길을 가다가 우연히 마주친 동네 아주머니께서 취업했냐고 물어보셔서 당황했어
15    어제 합격 통보를 받은 회사에서 문자를 잘못 발송했다고 연락이 왔어 너무 당혹스럽고...
16                              나 오늘 첫 출근 했는데 너무 당황스러웠어
17        이번에 직장을 이직했는데 글쎄 만나고 싶지 않은 사람을 만나서 아주 

In [109]:
# import jdk
# jdk.install('8')

In [110]:
import os
os.environ['JAVA_HOME'] = '/home/j-j10a506/.jdk/jdk8u402-b06'
os.environ['PATH'] = f"{os.environ.get('PATH')}:{os.environ.get('JAVA_HOME')}/bin"

#### Okt

In [111]:
from konlpy.tag import Okt
import time

In [112]:
okt = Okt()
start_time = time.time()
for sentence in test_sentences:
    okt.morphs(sentence, stem=True)
end_time = time.time()
print(f"time: {end_time-start_time}ms")

time: 0.496063232421875ms


In [113]:
for sentence in test_sentences:
    print(okt.morphs(sentence, stem=True))

['일', '은', '왜', '해도', '해도', '끝', '이', '없다', '화가', '나다']
['이번', '달', '에', '또', '급여', '가', '깎다', '물가', '는', '오르다', '월급', '만', '자꾸', '깎다', '너무', '화가', '나']
['회사', '에', '신입', '이', '들어오다', '말투', '가', '거슬리다', '그렇다', '애', '를', '매일', '보다', '하다', '생각', '하다', '스트레스', '받다']
['직장', '에서', '막내', '라는', '이유', '로', '나', '에게만', '온갖', '심부름', '을', '시키다', '일도', '많다', '데', '정말', '분하다', '섭섭하다']
['얼마', '전', '입사', '한', '신입사원', '이', '나르다', '무시', '하다', '것', '같다', '너무', '화가', '나']
['직장', '에', '다니다', '있다', '시간', '만', '버리다', '거', '같다', '진지하다', '진로', '에', '대한', '고민', '이', '생기다']
['성인', '인데', '도', '진로', '를', '아직도', '못', '정', '하다', '부모님', '이', '노', '여', '워', '하다', '나다', '섭섭하다']
['퇴사', '한', '지다', '얼마', '안', '돼다', '천천히', '직장', '을', '구', '해보다']
['졸업', '반', '이라서', '취업', '을', '생각', '하다', '하다', '지금', '너무', '느긋하다', '이래도', '되다', '싶다']
['요즘', '직장', '생활', '이', '너무', '편하다', '좋다', '것', '같다']
['취업', '하다', '하다', '나이', '인데', '취업', '하고', '싶다', '않다']
['면접', '에서', '부모님', '직업', '에', '대한', '질문', '이', '들어오다']
['큰일', '이야', '부장', '님', '께', 

## 불용어 제거

In [114]:
stopwords = set()
with open("../dataset/stopwords-2024326.txt", "r") as f:
    for line in f:
        stopwords.add(line.strip())

In [115]:
from konlpy.tag import Okt
from collections import Counter

In [116]:
all_df = pd.read_csv(os.path.join(DATA_PATH, "raw/sentimental_analysis.csv"))
test_sentences = all_df["sentence"].map(lambda x: preprocessing(x))
print(test_sentences)

0                                   일은 왜 해도 해도 끝이 없을까 화가 난다
1              이번 달에 또 급여가 깎였어 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나
2         회사에 신입이 들어왔는데 말투가 거슬려 그런 애를 매일 봐야 한다고 생각하니까 스트...
3          직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜 일도 많은 데 정말 분하고 섭섭해
4                      얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나
                                ...                        
130666           우리만 뒤처지는 것 같고 그래도 열심히 살다 보면 우리도 집을 살 수 있겠지
130667    나도 결혼했지만 아이도 생기지 않고 그 정도로 행복하지 않거든 친구보다 못사는 것 ...
130668                                 요새 집값이 너무 올라서 한숨만 나와
130669                      맞아 그 친구와 비교하게 되니 자존감이 낮아지는 기분이야
130670                         맞아 하지만 그렇다고 아무나하고 결혼할 수도 없잖아
Name: sentence, Length: 130671, dtype: object


In [117]:
words_counter = Counter()
for sentence in tqdm(test_sentences):
    tokens = okt.morphs(sentence, stem=True)
    tmp_counter = Counter(tokens)
    words_counter += tmp_counter

  0%|          | 0/130671 [00:00<?, ?it/s]

In [118]:
less_than_5 = {word: count for word, count in words_counter.items() if count <= 5}
print(less_than_5)

{'발송': 4, '령액': 1, '실업률': 2, '실습생': 2, '보육': 2, '푹푹': 2, '독자': 3, '정식': 5, '세태': 3, '삼고': 4, '월급쟁이': 5, '꾸물거리다': 3, '삭제': 5, '기혼': 1, '교외': 3, '백해무익': 1, '꽝': 1, '양립': 1, '진땀': 2, '거참': 5, '선행': 2, '시말서': 4, '골인': 3, '만족감': 4, '소인배': 3, '직렬': 1, '이단': 2, '울렁증': 1, '섣부르다': 4, '가주': 5, '유머': 3, '시내': 3, '입상하다': 4, '토박이': 1, '전시': 3, '모유': 3, '수유': 2, '반지하': 2, '수완': 3, '납': 4, '평안하다': 4, '신호등': 2, '측은하다': 4, '재롱': 3, '자궁경부암': 4, '홍수': 5, '슈퍼마켓': 4, '알파벳': 4, '자율': 2, '핑곗거리': 1, '툴툴거린': 3, '툴툴거렸더': 1, '버럭버럭': 1, '재우다': 1, '눕혔는데': 1, '슬그머니': 4, '까치': 1, '미납': 4, '성화니': 1, '색시': 1, '청소기': 1, '부스럭': 1, '꼴불견': 2, '헛웃음': 4, '지우': 3, '팔목': 4, '고수': 4, '염좌': 2, '돈다': 2, '힐끔힐끔': 3, '매입': 3, '적립': 4, '제이': 3, '쩝쩝': 5, '사례금': 2, '흑자': 2, '경운기': 3, '일억이': 3, '경로': 3, '영구': 4, '부쉈는데': 1, '무전': 2, '쾅': 1, '추태': 4, '들락날락': 4, '쏠리다': 5, '체감': 5, '과소': 5, '수평': 4, '오락': 5, '재수술': 4, '무자비하다': 1, '선명하다': 3, '손수건': 1, '보건': 3, '호실': 3, '순발력': 3, '실태': 5, '응답': 3, '부류': 3, '북적': 2, '장려상': 1, '쑥떡': 4, '흥이': 5

In [119]:
for word, _ in less_than_5.items():
    stopwords.add(word)

In [120]:
# 조사, 시간, 부사(많이, 뭔가, 등) 제거
stopwords_add = ['은','는','이','가','하','아','것','들','의','에','있','되','수','보','주','등','한', '도', '께', '이렇다', '라는', '하다', '다', '되다', '근데', '랑', '드', '너무', '나니', '에는', '거야', '쯤', '뭔가', '엔', '이야', '많이', '거야', '하고', '만', '요즘', '나다', '오늘', '들다']

In [121]:
for stopword in stopwords_add:
    stopwords.add(stopword)

In [122]:
import datetime
def today():
    today = datetime.datetime.today()
    return str(today.year) + str(today.month) + str(today.day)

In [123]:
with open(os.path.join(DATA_PATH, "stopwords-{}-new.txt".format(today())), "w") as f:
    for stopword in stopwords:
        f.write(stopword.strip() + "\n")

#### 불용어 제거

In [124]:
all_df = pd.read_csv(os.path.join(DATA_PATH, "raw/sentimental_analysis.csv"))

In [125]:
def remove_stopwords(tokens):
    token_removed = [token for token in tokens if not token in stopwords]
    return " ".join(token_removed)

In [126]:
for idx, row in tqdm(all_df.iterrows()):
    row["sentence"] = preprocessing(row["sentence"])
    #tokens = okt.morphs(row['sentence'], stem=True)
    all_df.loc[idx, "sentence"] = row["sentence"] #remove_stopwords(tokens)

0it [00:00, ?it/s]

In [127]:
all_df['sentence'] = all_df['sentence'].map(lambda x: x.replace(" ", ""))
all_df = all_df[all_df['sentence'].apply(lambda x: len(x) >= 10)]
all_df = all_df[all_df['sentence'].apply(lambda x: len(x) <= 100)]
all_df = all_df[all_df['cat1'] != '중립']

In [128]:
all_df.to_csv(os.path.join(DATA_PATH, "sentimental_analysis_preprocessed.csv"), index=False) 

In [129]:
X_length = all_df['sentence'].astype(str).apply(len)
print('문장 길이 최대/최소: {} {}'.format(np.max(X_length), np.min(X_length)))
print('문장 길이 평균값: {:.2f}'.format(np.mean(X_length)))
print('문장 길이 표준편차: {:.2f}'.format(np.std(X_length)))
print('문장 길이 중간값: {}'.format(np.median(X_length)))

문장 길이 최대/최소: 96 10
문장 길이 평균값: 26.08
문장 길이 표준편차: 8.63
문장 길이 중간값: 26.0


## 1. (train, valid, test) 분할
* train/valid/test = 7:2:1

In [130]:
from sklearn.model_selection import train_test_split
import pandas as pd
import os

In [131]:
DATA_PATH = "../dataset"

In [132]:
all_df = pd.read_csv(os.path.join(DATA_PATH, "sentimental_analysis_preprocessed.csv"))

In [133]:
emotion_to_idx = {0:'분노', 1:'당황', 2:'슬픔', 3:'기쁨', 4:'불안', 5:"느긋"}
idx_to_emotion = {"분노": 0, "당황":1, "슬픔":2, "기쁨":3, "불안":4, "느긋":5}

In [134]:
all_df["cat1_idx"] = all_df["cat1"].map(lambda row: idx_to_emotion[row])
all_df = all_df[["sentence", "cat1_idx"]]
all_df = all_df.rename(columns={"sentence":"data", "cat1_idx":"target"})
all_df.head(3)

,data,target
0,일은왜해도해도끝이없을까화가난다,0
1,이번달에또급여가깎였어물가는오르는데월급만자꾸깎이니까너무화가나,0
2,회사에신입이들어왔는데말투가거슬려그런애를매일봐야한다고생각하니까스트레스받아,0


In [135]:
X_train, X_test, y_train, y_test = train_test_split(all_df["data"], all_df["target"], test_size=0.1, shuffle=True, stratify=all_df["target"], random_state=34)

In [136]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, shuffle=True, stratify=y_train, random_state=34)

In [137]:
train_df = pd.DataFrame({'X': X_train, 'y': y_train})
valid_df = pd.DataFrame({'X': X_valid, 'y': y_valid})
test_df = pd.DataFrame({'X': X_test, 'y': y_test})

In [138]:
def to_tsv(df, path, filename):
    if not os.path.exists(path):
        os.makedirs(path)
    df.to_csv(os.path.join(path, filename), sep="\t", index=False)

In [139]:
to_tsv(train_df, os.path.join(DATA_PATH, "train"), "train.tsv")
to_tsv(valid_df, os.path.join(DATA_PATH, "test"), "valid.tsv")
to_tsv(test_df, os.path.join(DATA_PATH, "test"), "test.tsv")

## 2. train/valid 분할
* Stratified K-fold 교차 검증
  * K=5
* tsv 형태로 저장

In [140]:
from sklearn.model_selection import StratifiedKFold

In [141]:
def to_tsv(df, path, filename):
    if not os.path.exists(path):
        os.makedirs(path)
    df.to_csv(os.path.join(path, filename), sep="\t", index=False)

In [142]:
kfold = StratifiedKFold(n_splits = 5)
train_list, valid_list = [], []
i = 0
for train_idx, valid_idx in kfold.split(train_df["X"], train_df["y"]):
    # print(train_idx, test_idx)
    train, valid =  train_df.iloc[train_idx],  train_df.iloc[valid_idx]

    train_list.append(train)
    valid_list.append(valid)

    print('학습 레이블 데이터 분포 : \n', train["y"].value_counts())
    print('검증 레이블 데이터 분포: \n', valid["y"].value_counts())

    to_tsv(train, "../dataset/train/split_{}".format(i), "train.tsv")
    to_tsv(valid, "../dataset/test/split_{}".format(i), "valid.tsv")

    i += 1
    print(i, train.shape, valid.shape)
    print("----------------------------------------------------------------")

학습 레이블 데이터 분포 : 
 2    22951
4    11832
0    11809
1    11139
5     5881
3     5023
Name: y, dtype: int64
검증 레이블 데이터 분포: 
 2    5738
4    2958
0    2953
1    2784
5    1471
3    1255
Name: y, dtype: int64
1 (68635, 2) (17159, 2)
----------------------------------------------------------------
학습 레이블 데이터 분포 : 
 2    22951
4    11832
0    11810
1    11138
5     5882
3     5022
Name: y, dtype: int64
검증 레이블 데이터 분포: 
 2    5738
4    2958
0    2952
1    2785
5    1470
3    1256
Name: y, dtype: int64
2 (68635, 2) (17159, 2)
----------------------------------------------------------------
학습 레이블 데이터 분포 : 
 2    22951
4    11832
0    11810
1    11138
5     5882
3     5022
Name: y, dtype: int64
검증 레이블 데이터 분포: 
 2    5738
4    2958
0    2952
1    2785
5    1470
3    1256
Name: y, dtype: int64
3 (68635, 2) (17159, 2)
----------------------------------------------------------------
학습 레이블 데이터 분포 : 
 2    22951
4    11832
0    11810
1    11138
5     5882
3     5022
Name: y, dtype: int64
검증 레이블 데이터 분